In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [681]:
# 감성사전 txt 불러오기
def load_sentiment_dictionary(file_path):
    sentiment_dict = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            word = line.strip()  # 각 줄의 공백 및 개행문자 제거
            sentiment_dict.add(word)
    return sentiment_dict

# CSV 파일을 데이터프레임으로 변환
def load_csv_file(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return df

# 감성사전과 토큰을 비교하여 긍정어인 경우 'pos_word' 열에 추가
def add_word_column(df, sentiment_dict):
    word_column = []
    for token in df['token']:
        words = token.split()
        positive_words = [word for word in words if word in sentiment_dict]
        word_column.append(' '.join(positive_words))
    df['pos_word'] = word_column
    return df

# 데이터프레임을 csv 파일로 저장
def save_dataframe_to_csv(df, output_file):
    df.to_csv(output_file, index=False, encoding='utf-8')

In [682]:
# 감성사전 텍스트 파일 불러오기
sentiment_dict = load_sentiment_dictionary('text/pos_word.txt')

# csv 파일 불러오기
df = load_csv_file('data/충청남도_pos.csv')

In [683]:
# 감성사전과 토큰을 비교하여 긍정어인 경우 'pos_word' 열에 추가
df_with_words = add_word_column(df, sentiment_dict)

In [684]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_with_words["pos_word"])

feature_names = vectorizer.get_feature_names_out()

# TF-IDF 값이 가장 큰 단어 추출 
def get_top_keywords(tfidf_matrix, feature_names, top_n=20):
    keywords = []
    for i in range(tfidf_matrix.shape[0]):
        scores = {feature_names[col]: tfidf_matrix[i, col] for col in tfidf_matrix[i].nonzero()[1]}
        top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        keywords.append([kw for kw, _ in top_keywords])
    return keywords

# 상위 20개의 핵심 키워드 추출
top_keywords_per_review = get_top_keywords(tfidf_matrix, feature_names, top_n=20)

df_with_words['pos_keywords'] = [' '.join(kw_list) for kw_list in top_keywords_per_review]

In [685]:
df_with_words

,시도,시군구,code,title,content,full_content,token,Sentiment_Label,pos_word,pos_keywords
0,충청남도,계룡시,44250.0,군엑스포/휴일나들이/<b>충청남도여행</b>/<b>계룡시</b>여행/당일치기여행,어제는 어디 갈까 하다가 남편이 계룡 세계 군 문화 엑스포를 가자고 해서 호기롭게 ...,군엑스포/휴일나들이/<b>충청남도여행</b>/<b>계룡시</b>여행/당일치기여행어제...,군 엑스포 휴일 나들이 당일치기 어제 갈다 남편 세계 군 문화 엑스포 호기롭 나서다...,Positive,좋아하다 교훈,교훈 좋아하다
1,충청남도,계룡시,44250.0,"논산 가볼만한곳, 돈암서원, 유네스코 세계유산, 돈암서원...",2021. 3. 4. (금)양촌면에 위치한 쌍계사를 구경하고 승용차로 30분 정도 ...,"논산 가볼만한곳, 돈암서원, 유네스코 세계유산, 돈암서원... 2021. 3. 4....",돈암서원 유네스코 세계 유산 돈암서원 금 양촌면 위치 쌍계사 구경 승용차 걸리다 연...,Positive,아름답다 반기다 만족하다 좋다,반기다 만족하다 아름답다 좋다
2,충청남도,계룡시,44250.0,"은총의샘,서초3동/성 비오성당",월간 ''레지오 마리애''로 선교 활동​월간 '레지오 마리애'는 한국레지오 마리애 ...,"은총의샘,서초3동/성 비오성당월간 ''레지오 마리애''로 선교 활동​월간 '레지오 ...",은총 샘 서초동 비다 성당 월간 레 지오 마리 애로 선교 활동 월간 레 지오 마리 ...,Positive,은총 성숙하다 성숙하다 좋다 참으로,성숙하다 은총 참으로 좋다
3,충청남도,계룡시,44250.0,<b>충청남도</b> 가볼만한 곳 지역별 총정리 (도시별 <b>여행</b>지 추천),​\n​\n주말에 떠나고 싶은 가볼만한 충청남도 여행지 지역별 총정리\n​​블로그씨...,<b>충청남도</b> 가볼만한 곳 지역별 총정리 (도시별 <b>여행</b>지 추천)...,지역별 정리 도시 별 여행지 추천 주말 떠나다 여행지 지역별 정리 블로그 칼춤 한번...,Positive,추천 추천 가치 상위 추천 좋아하다 놀다 좋다 놀다 놀다 기대 좋다 추천,놀다 추천 상위 가치 기대 좋다 좋아하다
4,충청남도,계룡시,44250.0,공주 <b>여행</b>지 추천 계룡산국립공원 천년고찰 갑사 19년11월 단풍,​춘마곡 추갑사로 유명한 계룡산 갑사 단풍 모습공주 여행지 추천 계룡산국립공원 천년...,공주 <b>여행</b>지 추천 계룡산국립공원 천년고찰 갑사 19년11월 단풍​춘마곡...,여행지 추천 계룡산 국립공원 천년 고찰 갑사 년월 단풍 추다 곡 추 갑사 유명하다 ...,Positive,추천 유명하다 추천 유명하다 아름답다 아름답다 이루다 추천 아름답다 즐기다 아름답다...,이루다 아름답다 으쓱 추천 높다 유명하다 뛰어나다 반기다 조화 적당하다 이루어지다 ...
...,...,...,...,...,...,...,...,...,...,...
4287,충청남도,홍성군,44800.0,"오서산, 홍성 갈만한 곳이자 7월 여행지로 딱인 백패킹 <b>명소</b> 추천","안녕하세요, 적벽이입니다.\n하늘은 우중충하고비가 주륵주륵 오는 장마로 인해집과 사...","오서산, 홍성 갈만한 곳이자 7월 여행지로 딱인 백패킹 <b>명소</b> 추천안녕...",오서산 갈다 이자 여행지 딱이다 패킹 명소 추천 안녕하다 적벽 하늘 우중충하다 고비...,Positive,추천 멋지다 행복 굉장하다 좋다 멋지다 좋다 높다 높다 좋다 다양하다 높다 멋지다 ...,높다 멋지다 부드럽다 고백 유명하다 즐기다 힘차다 짜릿하다 상쾌하다 행복 기대하다 ...
4288,충청남도,홍성군,44800.0,"홍성(12경) 새조개 축제,죽도","홍성(12경) 새조개 축제, 죽도2023. 3. 4(1박2일)​코스 ▶​서울-김좌진...","홍성(12경) 새조개 축제,죽도홍성(12경) 새조개 축제, 죽도2023. 3. 4(...",홍 성경 조개 축제 죽도 홍 성경 조개 축제 죽도 박일 코스 김좌진 장군 생가지 한...,Positive,싱싱하다 풍부하다 뛰어나다 좋아하다 아름답다 낭만 최고 승리 유명하다 다양하다 좋아...,아름답다 뛰어나다 유명하다 싱싱하다 최고 전성 행복하다 즐기다 승리 사랑 시원하다 ...
4289,충청남도,홍성군,44800.0,<b>충청남도</b>세종시대한민국경기도대한민국경상<b>남도</b>진...,​\n오늘28명 방문로그아웃\n내 블로그글쓰기\n내 소식내가 남긴 글이웃 목록\n새...,<b>충청남도</b>세종시대한민국경기도대한민국경상<b>남도</b>진... ​\n오늘...,대한민국 대한민국 진 오늘 방문 로그아웃 블로그 글쓰기 소식 남기다 글 이웃 목록 ...,Positive,빠르다 완성 대단하다 추천 도움 추천 추천 부자 높다 발전 발전 착하다 착하다 웰빙...,착하다 발전 부자 추천 웰빙 대단하다 완성 행복 도움 빠르다 제대로 높다
4290,충청남도,홍성군,44800.0,"홍성의 맛집 #23, 추어어죽, 홍성의 새로운 <b>명소</b> 탄생예고...",어제 밤 KBS 생생정보통에서 충남 홍성군 광천에 있는 광천원조어죽이 전국 방송을 ...,"홍성의 맛집 #23, 추어어죽, 홍성의 새로운 <b>명소</b> 탄생예고... 어제...",추어 죽 새롭다 명소 탄생 예고 어제 밤 kbs 생생정보통 광천 광천 원조 죽이다 ...,Positive,충분하다 인정 더하다 칭찬 대단하다 칭찬 독특하다 믿음 신뢰 높이다,칭찬 믿음 신뢰 인정 대단하다 높이다 독특하다 더하다 충분하다


In [686]:
result_df = df_with_words.groupby(['시도','시군구','code'])['pos_keywords'].agg(lambda x: ' '.join(map(str, x))).reset_index()

In [ ]:
output_file_path = 'data/충청남도_pos_keywords.csv'
save_dataframe_to_csv(result_df, output_file_path)

In [688]:
data = pd.read_csv('data/충청남도_pos_keywords.csv')

In [689]:
data

,시도,시군구,code,pos_keywords
0,충청남도,계룡시,44250.0,교훈 좋아하다 반기다 만족하다 아름답다 좋다 성숙하다 은총 참으로 좋다 놀다 추천 ...
1,충청남도,공주시,44150.0,영웅 배우다 빠르다 깨끗하다 감사하다 좋다 즐기다 아름답다 여유롭다 편리 다양하다 ...
2,충청남도,금산군,44710.0,반기다 낭만 곱다 청결하다 수월하다 아름답다 신선하다 장점 맑은 휴식 좋다 뿌듯하다...
3,충청남도,논산시,44230.0,현명하다 튼튼하다 의욕 기대 중요하다 괜찮다 친하다 수익 좋다 적합하다 밝다 대단하...
4,충청남도,당진시,44270.0,다양하다 휴식 추천 감사 반기다 놀다 높다 멋지다 맛있다 유명하다 즐기다 젊다 훌륭...
5,충청남도,보령시,44180.0,제대로 아끼다 감사 훌륭하다 인정 믿다 감동 반갑다 건강하다 대상 친절하다 깨끗하다...
6,충청남도,부여군,44760.0,제대로 아끼다 감사 훌륭하다 인정 믿다 감동 반갑다 건강하다 대상 친절하다 깨끗하다...
7,충청남도,서산시,44210.0,은혜 안도 보람 상위 감사 훌륭하다 완전하다 밝다 뛰어나다 감동 발전 안전하다 가치...
8,충청남도,서천군,44770.0,제대로 아끼다 감사 훌륭하다 인정 믿다 감동 반갑다 건강하다 대상 친절하다 깨끗하다...
9,충청남도,아산시,44200.0,융통성 빠르다 고백 좋다 아름답다 당당하다 유익하다 예쁘다 희다 높이다 건강하다 기...


In [690]:
# 여러 개의 CSV 파일 읽기
files = ['data/강원특별자치도_pos_keywords.csv', 'data/경기도_pos_keywords.csv', 'data/경상남도_pos_keywords.csv', 'data/경상북도_pos_keywords.csv', 'data/광주광역시_pos_keywords.csv', 'data/대구광역시_pos_keywords.csv', 'data/대전광역시_pos_keywords.csv', 'data/부산광역시_pos_keywords.csv','data/서울특별시_pos_keywords.csv', 'data/세종특별자치시_pos_keywords.csv','data/울산광역시_pos_keywords.csv','data/인천광역시_pos_keywords.csv','data/전라남도_pos_keywords.csv','data/전라북도_pos_keywords.csv','data/제주특별자치도_pos_keywords.csv','data/충청남도_pos_keywords.csv','data/충청북도_pos_keywords.csv']

# 데이터프레임들을 수직으로 합치기
dfs = [pd.read_csv(file) for file in files]
pos_word_all = pd.concat(dfs, ignore_index=True)

In [691]:
pos_word_all

,시도,시군구,code,pos_keywords
0,강원특별자치도,강릉시,42150.0,할인 좋다 혜택 맑다 빼어나다 멋있다 특이하다 저렴하다 특별하다 편하다 추천 데이트...
1,강원특별자치도,고성군,42820.0,예쁘다 적당하다 실속 친구 좋다 부지런하다 즐거움 좋아지다 중요하다 재미있다 고르다...
2,강원특별자치도,동해시,42170.0,신기하다 추천 재미있다 시원하다 제대로 용이하다 서늘하다 선명하다 독특하다 화려하다...
3,강원특별자치도,삼척시,42230.0,즐기다 최적화 이루어지다 안전하다 원더풀 더없이 친구 분명하다 평온하다 미인 탁월하...
4,강원특별자치도,속초시,42210.0,예쁘다 적당하다 실속 친구 좋다 부지런하다 즐거움 좋아지다 중요하다 재미있다 고르다...
...,...,...,...,...
245,충청북도,청주시 상당구,43111.0,시원하다 사랑받다 소중하다 추천 도움 좋다 안전 중요하다 아름답다 즐기다 다양하다 ...
246,충청북도,청주시 서원구,43112.0,혜택 멋지다 완벽하다 기대하다 저렴하다 좋다 소중하다 아름답다 다양하다 가르침 존중...
247,충청북도,청주시 청원구,43114.0,만족하다 착하다 장점 저렴하다 깔끔하다 시원하다 이쁘다 추천 귀엽다 정겹다 좋다 유...
248,충청북도,청주시 흥덕구,43113.0,흥미롭다 대상 빠르다 할인 최고 높다 추천 혜택 멋지다 완벽하다 기대하다 저렴하다 ...


In [692]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(pos_word_all["pos_keywords"])

feature_names = vectorizer.get_feature_names_out()

# TF-IDF 값이 가장 큰 단어 추출 
def get_top_keywords(tfidf_matrix, feature_names, top_n=5):
    keywords = []
    for i in range(tfidf_matrix.shape[0]):
        scores = {feature_names[col]: tfidf_matrix[i, col] for col in tfidf_matrix[i].nonzero()[1]}
        top_keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        keywords.append([kw for kw, _ in top_keywords])
    return keywords

# 상위 5개의 핵심 키워드 추출
top_keywords_per_review = get_top_keywords(tfidf_matrix, feature_names, top_n=5)

pos_word_all['pos_top_keywords'] = [' '.join(kw_list) for kw_list in top_keywords_per_review]

In [693]:
pos_word_all = pos_word_all.groupby(['시도','시군구','code'])['pos_top_keywords'].agg(lambda x: ' '.join(map(str, x))).reset_index()

In [694]:
pos_word_all

,시도,시군구,code,pos_top_keywords
0,강원특별자치도,강릉시,42150.0,좋다 맛있다 추천 다양하다 즐기다
1,강원특별자치도,고성군,42820.0,좋다 즐기다 추천 다양하다 아름답다
2,강원특별자치도,동해시,42170.0,좋다 추천 즐기다 다양하다 아름답다
3,강원특별자치도,삼척시,42230.0,좋다 추천 다양하다 즐기다 맛있다
4,강원특별자치도,속초시,42210.0,좋다 맛있다 추천 다양하다 유명하다
...,...,...,...,...
245,충청북도,청주시 상당구,43111.0,좋다 추천 다양하다 즐기다 아름답다
246,충청북도,청주시 서원구,43112.0,좋다 다양하다 추천 맛있다 즐기다
247,충청북도,청주시 청원구,43114.0,좋다 다양하다 추천 즐기다 맛있다
248,충청북도,청주시 흥덕구,43113.0,좋다 추천 다양하다 좋아하다 높다


In [695]:
pos_word_all.to_csv('data/pos_top_keywords.csv', index=False, encoding='utf-8-sig')